In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import shutil

# Исходные папки
base_path = '/kaggle/input/drone-uav-bird/total_dataset'
splits = ['train', 'valid', 'test']

# Папки для сохранения
output_images = '/kaggle/working/clean_dataset/images'
output_labels = '/kaggle/working/clean_dataset/labels'

# Создаем выходные папки
os.makedirs(output_images, exist_ok=True)
os.makedirs(output_labels, exist_ok=True)

# Считаем общее количество файлов
total_files = sum(len(os.listdir(os.path.join(base_path, split, 'labels'))) for split in splits)
processed = 0

for split in splits:
    label_dir = os.path.join(base_path, split, 'labels')
    image_dir = os.path.join(base_path, split, 'images')
    
    for file in os.listdir(label_dir):
        if file.endswith('.txt'):
            processed += 1
            txt_path = os.path.join(label_dir, file)
            
            # Читаем строки
            with open(txt_path, 'r') as f:
                lines = f.readlines()
            
            # Фильтруем только строки с классом 1
            drone_lines = []
            for line in lines:
                parts = line.strip().split()
                if parts[0] == '1':
                    parts[0] = '0'  # заменяем class 1 -> class 0
                    drone_lines.append(' '.join(parts))
            
            # Если есть дрон в файле
            if drone_lines:
                # Сохраняем новый .txt
                out_txt_path = os.path.join(output_labels, file)
                with open(out_txt_path, 'w') as f:
                    f.write('\n'.join(drone_lines))
                
                # Копируем соответствующее изображение
                img_file = file.replace('.txt', '.jpg')
                src_img_path = os.path.join(image_dir, img_file)
                dst_img_path = os.path.join(output_images, img_file)
                
                if os.path.exists(src_img_path):
                    shutil.copy(src_img_path, dst_img_path)
                else:
                    print(f" Картинка {img_file} не найдена для {file}")
            



In [3]:

source_path = '/kaggle/input/aircraft-type-uavs/aircraft-type UAVs'

# Список файлов
files = os.listdir(source_path)

for file in files:
    src_file = os.path.join(source_path, file)
    if file.endswith('.jpg'):
        dst_file = os.path.join(output_images, file)
        shutil.copy(src_file, dst_file)
    elif file.endswith('.txt'):
        dst_file = os.path.join(output_labels, file)
        
        # Опционально: проверим, что class=0
        with open(src_file, 'r') as f:
            lines = f.readlines()
        
        fixed_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts[0] != '0':
                print(f" найден class != 0 в {file}: {line.strip()}")
            fixed_lines.append(line.strip())  # (оставляем как есть)
        
        # Сохраняем
        with open(dst_file, 'w') as f:
            f.write('\n'.join(fixed_lines))


In [4]:


# Исходная папка
source_path = '/kaggle/input/drone-dataset-uav/drone_dataset_yolo/dataset_txt'

# Создаем выходные папки, если не созданы
os.makedirs(output_images, exist_ok=True)
os.makedirs(output_labels, exist_ok=True)

# Проходим по всем файлам
for file in os.listdir(source_path):
    src_file = os.path.join(source_path, file)
    
    if file.endswith('.jpg'):
        dst_file = os.path.join(output_images, file)
        shutil.copy(src_file, dst_file)
    
    elif file.endswith('.txt'):
        dst_file = os.path.join(output_labels, file)
        shutil.copy(src_file, dst_file)


In [5]:
!pip install ultralytics

In [6]:
# Путь для сохранения yaml
yaml_path = '/kaggle/working/clean_dataset/data.yaml'

# Содержимое yaml
yaml_content = """
train: /kaggle/working/clean_dataset/images
val: /kaggle/working/clean_dataset/images

nc: 1
names: ['drone']
"""

# Запись в файл
with open(yaml_path, 'w') as f:
    f.write(yaml_content.strip())



In [ ]:
from ultralytics import YOLO

# Загружаем модель small
model = YOLO('yolov8s.pt')

# Обучаем
model.train(data='/kaggle/working/clean_dataset/data.yaml', epochs=50, imgsz=640)


Ultralytics 8.3.127 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/clean_dataset/data.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12

train: Scanning /kaggle/working/clean_dataset/labels... 21669 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21669/21669 [00:16<00:00, 1327.60it/s]

train: /kaggle/working/clean_dataset/images/pic_722.jpg: corrupt JPEG restored and saved


train: New cache created: /kaggle/working/clean_dataset/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1416.8±614.2 MB/s, size: 206.8 KB)


val: Scanning /kaggle/working/clean_dataset/labels.cache... 21669 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21669/21669 [00:00<?, ?it/s]

...

       7/50      5.75G      1.327     0.9572      1.201         33        640:   6%|▌         | 82/1355 [00:23<06:04,  3.49it/s]libpng warning: iCCP: known incorrect sRGB profile
       7/50      5.75G      1.304     0.9428      1.195         57        640:  38%|███▊      | 519/1355 [02:27<03:56,  3.53it/s]libpng warning: iCCP: known incorrect sRGB profile
       7/50      5.75G       1.29     0.9325      1.188         37        640:  63%|██████▎   | 851/1355 [04:02<02:22,  3.54it/s]libpng warning: iCCP: known incorrect sRGB profile
       7/50      5.75G      1.284     0.9243      1.187          8        640: 100%|██████████| 1355/1355 [06:25<00:00,  3.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   4%|▍         | 29/678 [00:08<05:12,  2.08it/s]libpng warning: iCCP: known incorrect sRGB profile
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   6%|▌         | 42/678 [00:12<02:52,  3.68it/s]lib

                   all      21669      38475      0.925      0.851      0.926      0.613

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       8/50      5.75G      1.267     0.8919      1.186         41        640:  17%|█▋        | 230/1355 [01:05<05:20,  3.51it/s]libpng warning: iCCP: known incorrect sRGB profile
       8/50      5.75G      1.255     0.8861      1.182         37        640:  29%|██▉       | 392/1355 [01:51<04:32,  3.54it/s]libpng warning: iCCP: known incorrect sRGB profile
       8/50      5.75G       1.26     0.8863      1.176         38        640:  83%|████████▎ | 1125/1355 [05:20<01:04,  3.54it/s]libpng warning: iCCP: known incorrect sRGB profile
       8/50      5.75G      1.256     0.8847      1.174         11        640: 100%|██████████| 1355/1355 [06:25<00:00,  3.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   4%|▍         | 28/678 [00:08<04:56,  2.19it/s]libpng warning: iCCP: known incorrect sRGB profile
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   6%|▌         | 42/678 [00:12<02:51,  3.70it/s]l

                   all      21669      38475      0.921      0.851      0.926      0.622

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       9/50      5.75G      1.242     0.8618       1.17         46        640:  25%|██▌       | 344/1355 [01:38<04:49,  3.49it/s]